# Testing the higher order signature kernel
***

The esig package implements the computation of the higher order signature kernel, which we can use to validate our implementation.

In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys

sys.path.append('..')  # add to path parent dir of gpsig
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# numerics
import numpy as np

# signatures
import gpsig
import esig

***
To do so, we simply compare the entries of the signature kernel matrix computed by gpsig with inner products of signature features computed by esig. First, generate some random data, the details of which is irrelevant for this comparison.

In [2]:
num_levels = 5
num_examples = 100
len_examples = 50
num_features = 3
X = np.random.randn(num_examples, len_examples, num_features)

***
### Validating the signature kernel
##### Computing signature features with esig

In [3]:
esig.is_library_loaded()
sigs = np.asarray([esig.tosig.stream2sig(x, num_levels) for x in X])

The sigs array contains signature features up to level $M=5$ flattened out into $(1 + d + d^2 + \dots + d^M)$ dimensions. Signatures are tensors in the truncated tensor algebra $\mathbf{S}_{\leq M}(\mathbf{x}) \in \prod_{m=0}^M (\mathbb{R}^d)^{\otimes m}$, but this space is analogous to $\mathbb{R}^{1+d+d^2+\dots+d^M}$ with the Euclidean inner product, which we can use on these flattened out tensors to recover the signature kernel.

In [4]:
K_esig = sigs @ sigs.T

##### Computing the signature kernel with gpsig

In gpsig, we first use a state-space embedding $x \mapsto \kappa(x, \cdot)$ from $\mathbb{R}^d$ into an RKHS $V$, i.e. with some abuse of notation $\kappa_{\mathbf{x}} = (\kappa(x_i, \cdot))_{i=1,\dots, l_{\mathbf x}}$ for $\mathbf{x} = (x_i)_{i=1,\dots,l_{\mathbf x}}$. To recover the same setting as in esig, we may use as state-space embedding the identity map, which specifies that the inner product of two observations is simply the Euclidean inner product. This variant of the signature kernel is called _SignatureLinear_ here.

We remark that esig uses the highest order signature features, which corresponds in our case to setting $D = M$, i.e. _order = num_levels_. Furthermore, the default setting is to normalize each signature level, which we have to turn off.

In [5]:
input_dim = num_features * len_examples
kern = gpsig.kernels.SignatureLinear(input_dim, num_features, num_levels,
                                     order=num_levels, normalization=False,
                                     lengthscales=1)
K_gpsig = kern.compute_K_symm(X.reshape([num_examples, -1]))
# print(kern.lengthscales)
# merge last two axes of the input since the kernel expects a 2d array

jitter: 1e-06
float dtype: <class 'numpy.float64'>
int dtype: <class 'numpy.int32'>
(3,)


2024-02-08 19:24:51.913781: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2024-02-08 19:24:51.919438: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3601565000 Hz
2024-02-08 19:24:51.920420: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x471e870 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-08 19:24:51.920464: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


##### Comparing the results

In [6]:
K_diff = K_esig - K_gpsig
print('2-norm: {}'.format(np.linalg.norm(K_diff, ord=2)))
print('Fro-norm: {}'.format(np.linalg.norm(K_diff, ord='fro')))
print('Inf-norm: {}'.format(np.linalg.norm(K_diff, ord=np.inf)))

2-norm: 7.4074146094254315e-09
Fro-norm: 2.022270488955288e-08
Inf-norm: 3.455966179899406e-08


### Validating the (augmented) signature vs tensor kernel
First, let us generate some sparse tensors of the form $\mathbf{z} = (z_{m,1} \otimes \dots \otimes z_{m, m})_{m=0,\dots,M}$, i.e. we generate the elements $z_{m,i} \in \mathbb{R}^d$ in the tensor products for each $0 \geq i \geq m$ and $0 \geq m \geq M$.

The gpsig kernel expects that the tensors are in $(M(M+1)/2, n_{\mathbf Z}, d)$ format, i.e. all $z_{m, i}$ are stacked together along the first axis.

In [7]:
num_tensors = 100
Z = np.random.randn(int(num_levels * (num_levels + 1) / 2), num_tensors,
                    num_features)

##### Computing the corresponding tensor features

The generated components are a low-dimensional representation of the generally high-dimensional tensors, which is feasible due to the sparsity constraint. Hence, next we build the actual tensors that take values in $\prod_{m=0}^M (\mathbb{R}^d)^{\otimes m}$, but we flatten the dimensions out, similarly to the signature features previously.

In [8]:
tens = [np.ones((100, 1))]
k = 0
for m in range(1, num_levels + 1):
    Zm = Z[k]
    k += 1
    for i in range(1, m):
        Zm = (Zm[..., None] * Z[k, :, None, :]).reshape([num_tensors, -1])
        k += 1
    tens.append(Zm)
tens = np.concatenate(tens, axis=1)

In [9]:
K_tens_vs_sig = tens @ sigs.T

##### Computing the tensors vs signatures kernel with gpsig

In [10]:
K_tens_vs_seq_gpsig = kern.compute_K_tens_vs_seq(Z,
                                                 X.reshape([num_examples, -1]))

##### Comparing the results

In [11]:
K_tens_vs_seq_diff = K_tens_vs_sig - K_tens_vs_seq_gpsig
print('2-norm: {}'.format(np.linalg.norm(K_tens_vs_seq_diff, ord=2)))
print('Fro-norm: {}'.format(np.linalg.norm(K_tens_vs_seq_diff, ord='fro')))
print('Inf-norm: {}'.format(np.linalg.norm(K_tens_vs_seq_diff, ord=np.inf)))

2-norm: 1.461843794310358e-11
Fro-norm: 4.0155780433586177e-11
Inf-norm: 8.395639738978389e-11


### Validating the (augmented) tensor vs tensor kernel
Finally, we validate the computation of tensor vs tensor inner product in gpsig.

##### Computing the tensor vs tensor kernel as inner product of tensor features

In [12]:
K_tens_vs_tens = tens @ tens.T

##### Computing the tensor vs tensor kernel with gpsig

In [13]:
K_tens_vs_tens_gpsig = kern.compute_K_tens(Z)

##### Comparing the results

In [14]:
K_tens_vs_tens_diff = K_tens_vs_tens - K_tens_vs_tens_gpsig
print('2-norm: {}'.format(np.linalg.norm(K_tens_vs_tens_diff, ord=2)))
print('Fro-norm: {}'.format(np.linalg.norm(K_tens_vs_tens_diff, ord='fro')))
print('Inf-norm: {}'.format(np.linalg.norm(K_tens_vs_tens_diff, ord=np.inf)))

2-norm: 9.164109304523378e-13
Fro-norm: 1.5978343852785568e-12
Inf-norm: 1.4150902671872245e-12
